In [22]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# CSV 파일 읽기
product_df = pd.read_csv('ProductDB.csv')

# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 브랜드별 색상 토너의 총 재고 계산
total_stock_by_color = product_df.groupby('color')['stock'].sum().reset_index()

# CSS 스타일 정의 (개선된 테이블 스타일)
table_style = {
    'border-collapse': 'collapse',
    'width': '100%',
    'margin': '20px 0',
    'font-size': '16px',
    'text-align': 'center',
    'border': '1px solid #ddd',
    'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',
}

th_td_style = {
    'border': '1px solid #ddd',
    'padding': '12px',
    'background-color': '#f8f9fa',  # 더 밝은 배경색
    'color': '#333',  # 텍스트 색상
    'font-weight': 'bold',
    'text-transform': 'uppercase',  # 대문자 변환
    'letter-spacing': '0.05em',  # 글자 사이 간격
}

td_style = {
    'border': '1px solid #ddd',
    'padding': '12px',
    'background-color': '#ffffff',  # 흰색 배경
    'color': '#333',
    'font-weight': 'normal',
}

# 레이아웃 정의
app.layout = html.Div(children=[
    html.H1(children='Toner Stock Dashboard'),

    html.Div(children='''This dashboard shows the toner stock levels by brand and color.'''),

    dcc.Dropdown(
        id='brand-selector',
        options=[
            {'label': maker, 'value': maker} for maker in product_df['maker'].unique()
        ],
        value=product_df['maker'].unique()[0]
    ),

    dcc.Graph(
        id='brand-stock-graph'
    ),

    dcc.Graph(
        id='total-stock-graph'
    ),

    # 표 추가 (브랜드별 색상 토너 재고량)
    html.H2(children='Toner Stock Table by Brand and Color'),
    html.Table(id='stock-table', children=[
        html.Tr([html.Th('Brand', style=th_td_style), 
                 html.Th('Black', style=th_td_style), 
                 html.Th('Red', style=th_td_style), 
                 html.Th('Green', style=th_td_style), 
                 html.Th('Blue', style=th_td_style)]),
        *[
            html.Tr([html.Td(maker, style=td_style)] + 
                    [html.Td(product_df[(product_df['maker'] == maker) & (product_df['color'] == color)]['stock'].sum(), style=td_style) 
                     for color in ['black', 'red', 'green', 'blue']])
            for maker in product_df['maker'].unique()
        ]
    ], style=table_style)
])

# 콜백 설정 - 선택된 브랜드의 색상별 재고 그래프 업데이트
@app.callback(
    Output('brand-stock-graph', 'figure'),
    [Input('brand-selector', 'value')]
)
def update_brand_stock_graph(selected_maker):
    filtered_df = product_df[product_df['maker'] == selected_maker]
    fig = px.bar(filtered_df, x='color', y='stock', title=f'{selected_maker} Toner Stock by Color', color='color',
                 color_discrete_map={'black': 'rgba(0, 0, 0, 0.6)', 'red': 'rgba(255, 0, 0, 0.6)', 'green': 'rgba(0, 255, 0, 0.6)', 'blue': 'rgba(0, 0, 255, 0.6)'})

    fig.update_layout(
        xaxis_title='Color',
        yaxis_title='Stock Level',
        title_x=0.5,
        template='plotly_white'
    )
    return fig

# 콜백 설정 - 색상별 총 토너 재고 그래프 업데이트
@app.callback(
    Output('total-stock-graph', 'figure'),
    [Input('brand-selector', 'value')]
)
def update_total_stock_graph(selected_maker):
    fig = px.bar(total_stock_by_color, x='color', y='stock', title='Total Toner Stock by Color', color='color',
                 category_orders={'color': ['black', 'red', 'green', 'blue']},
                 color_discrete_map={'black': 'rgba(0, 0, 0, 0.6)', 'red': 'rgba(255, 0, 0, 0.6)', 'green': 'rgba(0, 255, 0, 0.6)', 'blue': 'rgba(0, 0, 255, 0.6)'})

    fig.update_layout(
        xaxis_title='Color',
        yaxis_title='Total Stock Level',
        title_x=0.5,
        template='plotly_white'
    )
    return fig

# 서버 실행
if __name__ == '__main__':
    app.run_server(debug=True)
